In [ ]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ChromeOptions
from tqdm import tqdm
import xml.etree.ElementTree as ET

options = ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)
url = "https://app.pdok.nl/viewer/"
driver.get(url)

driver.implicitly_wait(5)

elements = driver.find_elements(By.CLASS_NAME, "ggc-dt-info")

hrefs = [
    element.get_attribute("href")
    for element in elements
    if element.get_attribute("href")
]

In [31]:
data = []
target_hrefs = []

for href in tqdm(hrefs):
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )

    driver.get(href)
    driver.implicitly_wait(1)
    elements = driver.find_elements(By.CSS_SELECTOR, "a[data-text]")
    ng_hrefs = [element.get_attribute("data-text") for element in elements]
    
    # Add source-target pairs to the data list
    for ng_href in ng_hrefs:
        data.append({
            'source_url': href,
            'target_url': ng_href
        })
    
    target_hrefs.extend(ng_hrefs)
    driver.quit()
    
    if len(target_hrefs) % 10 == 0:
        print(f"Processed {len(target_hrefs)} URLs")

url_mapping_df = pd.DataFrame(data)

Processed 10 URLs


Processed 20 URLs


Processed 30 URLs


Processed 40 URLs


Processed 50 URLs


...

Processed 260 URLs


Processed 270 URLs


Processed 280 URLs


Processed 290 URLs


100%|██████████| 262/262 [11:36<00:00,  2.66s/it]


In [34]:
url_mapping_df = url_mapping_df.drop_duplicates()

In [ ]:
df = pd.DataFrame()

for row in url_mapping_df.itertuples():
    try:
        namespaces = {
            "wms": "http://www.opengis.net/wms",
            "xlink": "http://www.w3.org/1999/xlink",
        }

        response = requests.get(row.target_url)
        root = ET.fromstring(response.content)

        layers = root.findall(".//wms:Layer", namespaces)
        titles = []
        abstracts = []
        source_urls = []
        target_urls = []

        authority_name = None
        authority_link = None

        for layer in layers:
            title = layer.find("wms:Title", namespaces)
            title_text = title.text if title is not None else "No Title"

            abstract = layer.find("wms:Abstract", namespaces)
            abstract_text = abstract.text if abstract is not None else "No Abstract"

            authority_url = layer.find("wms:AuthorityURL", namespaces)
            if authority_url is not None:
                authority_name = authority_url.get("name")
                authority_link = authority_url.find(
                    "wms:OnlineResource", namespaces
                ).get("{http://www.w3.org/1999/xlink}href", None)

            titles.append(title_text)
            abstracts.append(abstract_text)
            source_urls.append(row.source_url)
            target_urls.append(row.target_url)

        tmp_df = pd.DataFrame(data={"title": titles, "abstract": abstracts, "source_url": source_urls, "target_url": target_urls})
        tmp_df["authority"] = authority_name
        tmp_df["authority_url"] = authority_link
        df = pd.concat([df, tmp_df])
    except Exception as e:
        print(f"Error: {e}")

Error: not well-formed (invalid token): line 1, column 0
Error: not well-formed (invalid token): line 1, column 0
Error: not well-formed (invalid token): line 1, column 0
Error: not well-formed (invalid token): line 1, column 0


In [67]:
df.to_csv("data.csv", index=False)

In [68]:
df

,title,abstract,source_url,target_url,authority,authority_url
0,Aardkundige Waarden - Provincies (INSPIRE geha...,Deze nationale dataset bevat de Aardkundige wa...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/aardkundige...,provincies,https://www.bij12.nl/
1,Protected Sites,Deze nationale dataset bevat de Aardkundige wa...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/aardkundige...,provincies,https://www.bij12.nl/
2,Protected Sites – Nature Conservation,Deze nationale dataset bevat de Aardkundige wa...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/aardkundige...,provincies,https://www.bij12.nl/
0,Actueel Hoogtebestand Nederland (AHN) WMS,Web Map Service (WMS) van het Actueel Hoogtebe...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/rws/ahn/wms/v1_0?reque...,rws,http://www.rijkswaterstaat.nl
1,"Digital Terrain Model (DTM) 0,5m","Het AHN DTM is bedoeld als maaiveldbestand, wa...",https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/rws/ahn/wms/v1_0?reque...,rws,http://www.rijkswaterstaat.nl
...,...,...,...,...,...,...
4,Shoreline,De grens tussen een zeegebied en het vaste lan...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/kadaster/sr/wms/v1_0?r...,kadaster,https://www.kadaster.nl
0,Zwemwaterkwaliteit (Provinciaal en Rijkswatere...,Menselijke gezondheid en veiligheid. De datase...,https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/zwemwaterkw...,provincies,https://www.bij12.nl/
1,Health determinant measure,Zwemwaterkwaliteit (Provinciaal en Rijkswateren),https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/zwemwaterkw...,provincies,https://www.bij12.nl/
0,Zwemwaterlocaties (Provinciaal en Rijkswateren...,"Gebiedsbescherming, Deze dataset bevat Zwemwat...",https://nationaalgeoregister.nl/geonetwork/srv...,https://service.pdok.nl/provincies/zwemwater-p...,provincies,https://www.bij12.nl/


In [ ]:
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

In [69]:
load_dotenv(".env")
key = os.environ.get("GEMENI_API_KEY")

docs = [
    Document(
        page_content=row["abstract"],
        metadata={
            "title": row["title"],
            "authority": row["authority"],
            "authority_url": row["authority_url"],
            "target_url": row["target_url"],
            "source_url": row["source_url"],
        },
    )
    for _, row in df.fillna("").iterrows()
]

In [70]:
docs

[Document(metadata={'title': 'Aardkundige Waarden - Provincies (INSPIRE geharmoniseerd) WMS', 'authority': 'provincies', 'authority_url': 'https://www.bij12.nl/', 'target_url': 'https://service.pdok.nl/provincies/aardkundige-waarden/wms/v1_0?request=GetCapabilities&service=WMS', 'source_url': 'https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/33f3b24a-0b34-4153-9930-726d575a04d1'}, page_content='Deze nationale dataset bevat de Aardkundige waarden. De dataset Aardkundige waarden valt binnen het INSPIRE-thema Beschermde gebieden.'),
 Document(metadata={'title': 'Protected Sites', 'authority': 'provincies', 'authority_url': 'https://www.bij12.nl/', 'target_url': 'https://service.pdok.nl/provincies/aardkundige-waarden/wms/v1_0?request=GetCapabilities&service=WMS', 'source_url': 'https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/33f3b24a-0b34-4153-9930-726d575a04d1'}, page_content='Deze nationale dataset bevat de Aardkundige waarden. De da

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=key, model="models/embedding-001"
)

text_splitter_docx = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splitted_docs = text_splitter_docx.split_documents(docs)

In [76]:
splitted_docs

[Document(metadata={'title': 'Aardkundige Waarden - Provincies (INSPIRE geharmoniseerd) WMS', 'authority': 'provincies', 'authority_url': 'https://www.bij12.nl/', 'target_url': 'https://service.pdok.nl/provincies/aardkundige-waarden/wms/v1_0?request=GetCapabilities&service=WMS', 'source_url': 'https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/33f3b24a-0b34-4153-9930-726d575a04d1'}, page_content='Deze nationale dataset bevat de Aardkundige waarden. De dataset Aardkundige waarden valt binnen het INSPIRE-thema Beschermde gebieden.'),
 Document(metadata={'title': 'Protected Sites', 'authority': 'provincies', 'authority_url': 'https://www.bij12.nl/', 'target_url': 'https://service.pdok.nl/provincies/aardkundige-waarden/wms/v1_0?request=GetCapabilities&service=WMS', 'source_url': 'https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/33f3b24a-0b34-4153-9930-726d575a04d1'}, page_content='Deze nationale dataset bevat de Aardkundige waarden. De da

In [100]:
persist_directory = "./chroma_db"
vectorstore = Chroma.from_documents(
    documents=splitted_docs, embedding=embeddings, persist_directory=persist_directory
)

E0000 00:00:1738620048.491158 1987097 tcp_posix.cc:593] recvmsg encountered uncommon error: Can't assign requested address
